In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import colors

from googletrans import Translator
translator = Translator()
import pandas as pd

import json
import numpy as np
import time
import glob
import re
import ast

import geopandas
import seaborn as sns

from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


from word2number import w2n
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk import stem
stemmer = stem.snowball.EnglishStemmer()
import string

english_words = words.words()
stop = stopwords.words('english')

import en_core_web_sm
nlp = en_core_web_sm.load()

import pytextrank
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
from sklearn.decomposition import PCA
from wordcloud import WordCloud
from matplotlib import cm

from matplotlib import colors
import matplotlib.patches as mpatches
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error,mean_absolute_error


In [ ]:
raw_data_file = 'china_data.csv'

In [ ]:
china_data = pd.read_csv(raw_data_file)

##modity dtyeps
china_data['first_sentence_list'] = [ast.literal_eval(i) for i in china_data['first_sentence_list']]
china_data['first_sentence_pos_list'] = [ast.literal_eval(i) for i in china_data['first_sentence_pos_list']]
china_data['first_sentence_useful_list'] = [ast.literal_eval(i) for i in china_data['first_sentence_useful_list']]

# First clause reference

In [ ]:
first_sentence_list = []
first_sentence_pos_list = []
null_cnt = china_data['first_sentence_list'].isnull().sum()

for i, fs, fsp in zip(china_data['translated_company_desc'].values,
                       china_data['first_sentence_list'].values,
                       china_data['first_sentence_pos_list'].values):
    if True:
        try:
            ##get all clauses from the first sentence
            first_period = ' '.join(
        [i.text for i in nlp(i)][:[(i.pos_=='PUNCT') and (i.text=='.') for i in nlp(i)].index(True)]
    ).lower().replace('electricity business', 'online commerce')  #fix translation might need deletion
            first_period = first_period.replace('electric business', 'online commerce') # fix translation
            first_period = first_period.replace('electronic business', 'online commerce')
            first_period = first_period.replace('us - pupil','cosmetic contact lenses')
        except: 
            #some texts have no period
            print('no period detected')
            first_period = i.lower().replace('electricity business', 'online commerce') # fix translation
            first_period = first_period.replace('electric business', 'online commerce')
            first_period = first_period.replace('electronic business', 'online commerce')
            first_period = first_period.replace('us - pupil','cosmetic contact lenses')
            
        texts = first_period.split(',') # first clause only 
        list_useful = []
        list_useful_pos = []
        for text in texts:
            pos = [i.pos_ for i in nlp(text)]
            text = [i.text for i in nlp(text)]
            # skip clauses with no verb or aux 
            if ('VERB' not in pos) and ('AUX' not in pos):
                continue
            list_useful.append(text)
            list_useful_pos.append(pos)
        try:
            subset = list_useful[0]
            subset_pos = list_useful_pos[0]
            
            min_words = 15 ##first clause needs at least 15 words. If not, refer to the second, the third, .. if available
            recur = 1
            
            while len(subset)< min_words and len(list_useful)>= recur+1:
                subset = list(itertools.chain.from_iterable(list_useful[:recur+1]))
                subset_pos = list(itertools.chain.from_iterable(list_useful_pos[:recur+1]))
                recur+=1
                print('extending...')
                print(subset)
        except:
            # extract keywords from the whole text if no period detected.
            subset = list(np.array([i.text for i in nlp(first_period)])[
    [i.pos_ in ['NOUN','ADJ','PROPN'] for i in nlp(first_period)]])
            subset_pos = list(np.array([i.pos_ for i in nlp(first_period)])[
    [i.pos_ in ['NOUN','ADJ','PROPN'] for i in nlp(first_period)]]) 
    
    else:
        print('done already')
        subset = fs#.astype(list)
        subset_pos = fsp#.astype(list)
        
    print('finally..')
    print(subset)
    print(subset_pos)
    first_sentence_list.append(subset)
    first_sentence_pos_list.append(subset_pos)
    
china_data['first_sentence_list'] = first_sentence_list
china_data['first_sentence_pos_list'] = first_sentence_pos_list  
print(china_data['first_sentence_list'].isnull().sum())

china_data['first_sentence_list'] = china_data['first_sentence_list'].astype(object)
china_data['first_sentence_pos_list']= china_data['first_sentence_pos_list'].astype(object)

# Keyword Extraction

In [ ]:
first_sentence_useful_list = []
for i, j,u in zip(china_data.first_sentence_pos_list,
                  china_data.first_sentence_list,
                  china_data.first_sentence_useful_list):
    if True:
        ### filter company name by the position of first verb/aux
        try:
            index1 = i.index('VERB')
        except:
            index1 = 0
        try:
            index2 = i.index('AUX')
        except:
            index2 = 0
        index_position = min(index1,index2)
        ## keyword
        useful_list = [y for x, y in zip(i[index_position+1:],
                       j[index_position+1:]) if x in ['NOUN','ADJ','PROPN']] 
        
        ## might not needed
        if 'diagnostic' in j and 'diagnostic' not in useful_list:
            #print(j)
            useful_list.append('diagnostic') #correct
        ##--------------------    
        print(index_position, useful_list)
    else:
        useful_list = u
        
    first_sentence_useful_list.append(useful_list)  
    
print(china_data['first_sentence_useful_list'].isnull().sum())
china_data['first_sentence_useful_list'] = first_sentence_useful_list

# Sector Assignment

In [ ]:
top_k_list = []
cat_list = []

custom_stopwords = ['invest','investor','china',"investment", "capital",
                   'business','market','chinese', 'investments','financing','company',
                   'development', 'us','dollar','yuan','billion','million','round',
                   'rmb','investors','dollars','investor','valuation','share','stock',
                   'acquisition',"stocks",'shares','angel','venture','millions','billions','first',
                   'second','third','industry','company','products',
                   'services','brand','developer','developers','platform','2nd','results','announcement',
                    'provider','providers',
                    'developer','developers',
                   'object','total','tens','number','brands', 'enterprise',
                    'price', 'equity', 'partnership', 'people','creation','companies','production',
                   'more','issue','product','delisting','group','strategic', 'cooperation', 'agreement',
                   'subsidiary','world','today','name','results', 'announcement','dry','hereinafter',
                   'chain','enterprise','innovative','innovation','enterprises',
                   'technology',"tech",'development','prospectus', 'cornerstone', 'amount',
                   'fiscal', 'year', 'earnings','users','user','design','technologies','technological','high'
                   ,'ai','pupil','degree','electronic',
                   'international','global','institutions'] 


target_sector = ['manufacturer','automobile','semiconductor','hardware', 
                 'buildings','estate',
                 'financial', 'payment',
                 'information',"internet", 'software',
                 "healthcare","medical","pharmaceutical",'biotech',
                 "commerce","retail",
                 'education',
                 'game', 
                 'food',
                 'drink'
                ]


for corpus,top, cat in zip(china_data['first_sentence_useful_list'].values,
                  china_data['top_one'].values,
                  china_data['cat_list'].values):
    
    RUN_ALL = True
    if RUN_ALL:
        ## filter stopwords
        corpus = [i for i in corpus if i not in custom_stopwords]
        print(corpus)
        if len(corpus)>=1:
            list_top_k = []
            list_term = []
            list_simi = []
            for term in target_sector:
                for top_k in corpus:
                    top_k = top_k.lower()
                    if top_k in custom_stopwords:
                        continue
                    try:
                        ## similarity = 1/cosine_distance
                        list_simi.append(word2vec_model.similarity(top_k,term.lower()))
                        list_top_k.append(top_k)
                        list_term.append(term)
                    except KeyError:
                        pass
            tmp = pd.DataFrame({'top_k':list_top_k,
                                'term':list_term,
                                'simi':list_simi})  
            if len(tmp)!=0:
                cat = tmp.groupby('term').mean().sort_values('simi').index[-1]  #mean, median
                top_k = tmp[tmp['term']==cat].sort_values('simi').top_k.values[-1]
                top_k_list.append(top_k+' ')
                
                
                ###Might not needed
                general_list = ['supplier','operators','robot',"provider",'developer'] #manufacturer
                
                if cat in general_list: #  too general
                    print('detected')
                    #selected_cat = sorted(tmp.groupby('term').mean().sort_values('simi').index[-2:])
                    #selected_cat = tmp.groupby('term').mean().sort_values('simi').index[-2:]
                    #cat = '-'.join(selected_cat)
                    cat = tmp.groupby('term').mean().sort_values('simi').index[-2]
                    if cat in general_list:
                        cat = tmp.groupby('term').mean().sort_values('simi').index[-1]
                        
                if cat=='hardware':
                    if 'hardware' in corpus:
                        cat = 'hardware'
                    else: 
                        cat = tmp.groupby('term').mean().sort_values('simi').index[-2]
                        
                ##=================================
                cat_list.append(cat+' ')
                print(top_k, cat)
                
            else:
                top_k_list.append(' ')
                cat_list.append(' ')
                print('None')
        else:
            top_k_list.append(' ')
            cat_list.append(' ')
            print('None')
        
china_data['top_one'] = top_k_list
china_data['cat_list'] = cat_list   
print(china_data['top_one'].isnull().sum())

# Visualize Sector Assignment

In [ ]:
proxy_cols = ['Electronics',
              'RE',
               'IT',
              'Biotech',
             'Education'
             ]


proxy_sectors = [['hardware'], #semiconductor
                [ 'buildings','estate'],
                ['information',"internet", 'software'],
               [ "healthcare","medical","pharmaceutical"],
               [ 'education']]
                
    
dict_sector_cat = {}
chosen = [item for sublist in proxy_sectors for item in sublist]

for i, j in zip(proxy_cols,proxy_sectors):
    for ji in j:
        dict_sector_cat [ji] = i  

key_words = china_data['top_one']
sectors = china_data['cat_list']

vec_list = []
key_word_ = []
key_sector = []
for i,j in zip(key_words,sectors):
    try:
        vec_list.append(word2vec_model.get_vector(i.strip()))
        key_word_.append(i.strip())
        key_sector.append(j.strip())
    except:
        #print(i)
        pass
print(len(key_word_))
print(len(key_sector))

tmp_df = pd.DataFrame(vec_list)
tmp_df['sector'] = key_sector
tmp_df['word'] = key_word_

print(tmp_df.shape)
tmp_df = tmp_df.drop_duplicates()

tmp_df = tmp_df[[i in chosen for i in tmp_df["sector"]]]
tmp_df['sector'] = tmp_df['sector'].apply(lambda x:dict_sector_cat[x])    
tmp_df_subset = tmp_df.groupby('sector').head(25)

X = np.array(tmp_df_subset.iloc[:,:-2].values)
nkmeans= len(tmp_df_subset['sector'].unique())
kmeans = KMeans(n_clusters=nkmeans, random_state=0).fit(X)
kmeans.labels_
kmeans.cluster_centers_

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
print(pca.explained_variance_ratio_)

In [ ]:
le = LabelEncoder()
fig, ax = plt.subplots(figsize=(16,10))
ax.scatter(X_pca[:,0],X_pca[:,1], c= le.fit_transform(
    tmp_df_subset['sector']), cmap='rainbow',s = 1)

colors=cm.Dark2(np.linspace(0,1,len(tmp_df_subset['sector'].unique())))
dict_ ={i:j for i, j in zip(tmp_df_subset['sector'].unique(),
                            colors)}
for i, txt in enumerate(tmp_df_subset['sector'].values):
    ax.annotate(txt, (X_pca[:,0][i], X_pca[:,1][i]),color=dict_[txt],fontsize=15)

# Keyword wordcloud 

In [ ]:
word_cld = [i for i in china_data['first_sentence_useful_list'].values]
word_cld = [item for sublist in word_cld for item in sublist]
text = ' '.join(word_cld)

viridis = cm.get_cmap('viridis')
wordcloud = WordCloud(max_font_size=25,
                     background_color="white", 
               stopwords=custom_stopwords, contour_width=3,
                     contour_color='white'
                     ).generate(text)
plt.figure(figsize = (16,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Model 

In [ ]:
official_df = pd.read_excel('NFFAI_per_sector/fixed_investment202005.xls').iloc[1:,:]
official_df.columns = [i.replace('年','').replace('月','') for i in official_df.iloc[0,:].values]
official_df = official_df.set_index('指标')
official_df.columns = [i[:4]+'-'+i[4:].zfill(2) for i in official_df.columns]
official_df.index

In [ ]:
official_cols = ['医药制造业固定资产投资额累计增长(%)',
                '制造业固定资产投资额累计增长(%)',
                '信息传输、软件和信息技术服务业固定资产投资额累计增长(%)',
                '房地产业固定资产投资额累计增长(%)',
                '计算机、通信和其他电子设备制造业固定资产投资额累计增长(%)',
                 '金融业固定资产投资额累计增长(%)',
                #'住宿和餐饮业固定资产投资额累计增长(%)',
                 '教育固定资产投资额累计增长(%)',
                '批发和零售业固定资产投资额累计增长(%)',
                '汽车制造业固定资产投资额累计增长(%)',
                '食品制造业固定资产投资额累计增长(%)',
                '酒、饮料和精制茶制造业固定资产投资额累计增长(%)',
                #'文化艺术业固定资产投资额累计增长(%)',
                 '文化、体育和娱乐业固定资产投资额累计增长(%)',
                 #'交通运输、仓储和邮政业固定资产投资额累计增长(%)'
                 #'科学研究和技术服务业固定资产投资额累计增长(%)',
                 #'电气机械和器材制造业固定资产投资额累计增长(%)',
                 #'化学纤维制造业固定资产投资额累计增长(%)'
                ] #'纺织服装、服饰业固定资产投资额累计增长(%)'


proxy_cols = ['Medical',
              'Manufacturing',
               'IT',
              'RE',
              #'estate',
              'Electronics',
             'financial',
             #'catering',
             'education',
             'retail&wholesale',
             'automotive',
             'food',
             'drink',
             #'culture',
              'entertainment',
              #'logistics'
              #'science research'
             #'machinery',
             ] #"clothing" 'material'


proxy_sectors = [['pharmaceutical','medical','healthcare'],
                 ['manufacturer','automobile','semiconductor','hardware'],#'food','drink'], #machinery
                 ['information',"internet","software"],
                 ['buildings','estate'],
                 ['semiconductor','hardware'],
                 ['financial','payment'],
                ['education'],
                ['commerce','food','drink'],
                 ['automobile'],
                 ['food'],
                 ['drink'],
               [ 'game'],
               # ['logistics'],
                ]

if len(proxy_sectors) == len(proxy_cols) and len(proxy_cols) == len(official_cols):
    list_df = []
    for official_col, proxy_col, proxy_sector in zip(
official_cols,
    proxy_cols,
    proxy_sectors
): 
        print(proxy_sector)
        official_sub = official_df.ix[official_col].to_frame()
        official_sub.columns = ['official']
        tmp = china_data[[i.strip() in proxy_sector
                  for i in china_data['cat_list']]].groupby(
'date').count()["title_url"].to_frame()
        tmp = pd.DataFrame({'date':pd.date_range(tmp.index[0],
              tmp.index[-1])}).set_index('date').join(
        tmp).fillna(0)
        tmp = tmp.rolling(7*4).sum().dropna()
        tmp = tmp.reset_index()
        tmp.date = list(map(lambda x:str(x)[:7],tmp.date))
        result = tmp.groupby('date').mean().pct_change(12).dropna().join(
        official_sub
).dropna()
        result['sector'] = [proxy_col]*len(result)
        list_df.append(result.reset_index(drop=False))
else:
    print('ERROR!')

In [ ]:
colors = iter([plt.cm.tab20(i) for i in range(len(pd.concat(list_df,axis=0)['date'].unique()))])
result_df = pd.concat(list_df,axis=0)
result_df[['title_url','official']] = result_df[['title_url','official']].astype(float)
#result_df = result_df.groupby('sector').mean().reset_index()
result_df.plot.scatter(x='title_url',y = 'official', figsize=(16,8))
months = pd.concat(list_df,axis=0)['date'].unique()
#months = ['2020-03']
colors = [plt.cm.Dark2(i) for i in range(len(months))]
patch_list=[]
for mo, color in zip(months,colors): 
    tmp_df = result_df[result_df['date']==mo]
    patch = mpatches.Patch(color=color, label=mo)
    patch_list.append(patch)
    for label, x, y in zip(tmp_df['sector'], tmp_df["title_url"].astype(float), 
                       tmp_df['official'].astype(float)):
        plt.annotate(
        label,
        xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='right', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc=color, alpha=1),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
        
plt.legend(handles=patch_list)
plt.ylabel('China Official Fixed Income Investment Monthly YoY by sector.')
plt.xlabel('Alternative source on PE deals Monthly YoY by Sector.')
print('corr: '+ str(result_df.corr().iloc[0,-1]))
plt.show()

## Model Performance

In [ ]:
## need modification here. 
## by sector

lm = linear_model.LinearRegression()
cv_results = cross_validate(lm, result_df['title_url'].values.reshape(len(result_df),-1), 
                            result_df['official'], cv=5,scoring = 'neg_mean_squared_error'
                           ) #neg_mean_absolute_error  #neg_mean_squared_error
print(cv_results['test_score'])
print()
print(np.mean(cv_results['test_score']))

## Market Expectation

In [ ]:
#market_expectation 
expected = pd.DataFrame({'date': ['2020-02', '2020-03', '2020-04','2020-05'],
'expected': [-1.8,-15,-9.5,-5.9],
'true': [-24.5,-16.1, -10.3,-6.3]}).set_index('date')

print(expected.corr())
#1. http://economy.caixin.com/2020-03-16/101528938.html
#2. https://finance.sina.com.cn/money/bond/market/2020-04-16/doc-iircuyvh8165813.shtml
#3.https://cn.wsj.com/articles/%E4%B8%AD%E5%9B%BD1-4%E6%9C%88%E4%B8%8D%E5%90%AB%E5%86%9C%E6%88%B7%E5%9B%BA%E5%AE%9A%E8%B5%84%E4%BA%A7%E6%8A%95%E8%B5%84%E5%90%8C%E6%AF%94%E4%B8%8B%E9%99%8D10-3%EF%BC%8C%E4%B8%8D%E5%8F%8A%E9%A2%84%E6%9C%9F-11589508909
#4.https://cn.reuters.com/article/china-may-industry-consumption-poll-0612-idCNKBS23J1MU
expected.plot.scatter(x='expected',y = 'true',s = 50)

mean_squared_error(expected['expected'].values, 
                   expected.true)